# Aadhaar 360: National Strategic Intelligence Grid
## Notebook 2: The 5 Strategic Solutions (Massive Dynamics Edition)

This notebook implements the **5 Strategic Solutions** using advanced **Dynamic Data Visualization (Plotly)** to uncover hidden patterns, massive trends, and actionable intelligence.

### The 5-Module Vision:
1.  **👻 The Ghost Child Finder**: Dynamic Scatter Analysis of Child vs. Adult Enrolment Gaps.
2.  **🚧 The Border Watch**: 3D Risk Heatmaps of 18+ Enrolment Spikes.
3.  **📡 The Migration Radar**: Interactive Flow Analysis of Demographic Updates.
4.  **🛡️ The Integrity Shield**: Machine Learning Anomaly Detection with Time-Series Zoom.
5.  **🎓 Youth Awareness Tracker**: Correlation Engines for Biometric Update Lag.

---

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest

# Load Preprocessed Data
try:
    enrolment = pd.read_csv('d:/UIDAI/processed_data/enrolment_clean.csv', parse_dates=['date'])
    demographic = pd.read_csv('d:/UIDAI/processed_data/demographic_clean.csv', parse_dates=['date'])
    biometric = pd.read_csv('d:/UIDAI/processed_data/biometric_clean.csv', parse_dates=['date'])
    print("DATA LOADED: Ready for Massive Insight Extraction.")
except FileNotFoundError:
    print("CRITICAL ERROR: Data not found. Run Notebook 1 first.")

CRITICAL ERROR: Data not found. Run Notebook 1 first.


## 1. 👻 The "Ghost Child" Finder (Dynamic Gap Analysis)
**Insight**: Identifying districts where child enrolment is statistically decoupled from adult population growth.
**Visualization**: Interactive Scatter Plot with Hover-Data for every district.

In [2]:
district_ages = enrolment.groupby(['state', 'district'])[['age_0_5', 'age_5_17', 'age_18_greater']].sum().reset_index()
district_ages['Total'] = district_ages[['age_0_5', 'age_5_17', 'age_18_greater']].sum(axis=1)
district_ages['Child_Share'] = district_ages['age_0_5'] / district_ages['Total']

# Logic: Z-Score for Anomaly Detection
district_ages['Z_Score'] = (district_ages['Child_Share'] - district_ages['Child_Share'].mean()) / district_ages['Child_Share'].std()
ghost_child_districts = district_ages[district_ages['Z_Score'] < -1.5] # Bottom 1.5 Sigma

print(f"INSIGHT: Discovered {len(ghost_child_districts)} districts with massive 'Ghost Child' gaps.")

# Dynamic Visualization
fig = px.scatter(district_ages,
                 x='age_18_greater',
                 y='age_0_5',
                 color='Z_Score', # Color by Deviation
                 size='Total',
                 hover_name='district',
                 hover_data=['state', 'Child_Share'],
                 color_continuous_scale='RdYlGn',
                 title='Ghost Child Finder: Child vs Adult Activity (Red = Alert)',
                 template='plotly_dark')

fig.add_shape(type="line", x0=0, y0=0, x1=district_ages['age_18_greater'].max(), y1=district_ages['age_0_5'].max(),
              line=dict(color="White", width=1, dash="dash"))

fig.show()

NameError: name 'enrolment' is not defined

## 2. 🚧 The Border Watch (Adult Enrolment Tripwire)
**Insight**: Pinpointing border districts with sudden, unexplained surges in 18+ new enrolments (National Security Risk).
**Visualization**: Bar Chart of Risk Intensity.

In [ ]:
adult_enrolments = enrolment[enrolment['age_18_greater'] > 0].copy()
adult_alerts = adult_enrolments.groupby(['state', 'district'])['age_18_greater'].sum().reset_index()

# Top Risk Districts
top_risk = adult_alerts.sort_values('age_18_greater', ascending=False).head(20)

fig = px.bar(top_risk,
             x='age_18_greater',
             y='district',
             color='age_18_greater',
             orientation='h',
             color_continuous_scale='hot',
             hover_data=['state'],
             title='Border Watch: Top 20 Districts with Adult Enrolment Spikes (Security Alert)',
             template='plotly_dark')

fig.update_layout(yaxis=dict(autorange="reversed")) # Highest on top
fig.show()

## 3. 📡 The Migration Radar (Urban Shift Detection)
**Insight**: Correlating high-volume Demographic Updates with specific urban centers to predict service collapse.
**Visualization**: Hierarchical Treemap of Update Intensity.

In [ ]:
migration_data = demographic.groupby(['state', 'district'])['Total_Demo_Updates'].sum().reset_index()

# Treemap: State -> District drill-down
fig = px.treemap(migration_data,
                 path=[px.Constant("India"), 'state', 'district'],
                 values='Total_Demo_Updates',
                 color='Total_Demo_Updates',
                 color_continuous_scale='Viridis',
                 title='Migration Radar: Where is the population moving? (Update Volume)',
                 template='plotly_dark')
fig.show()

## 4. 🛡️ The Integrity Shield (Operational Anomalies)
**Insight**: Using Machine Learning (Isolation Forest) to detect fraud-like patterns (e.g., 1000 enrolments in 1 day).
**Visualization**: Anomaly Timeline Zoom.

In [ ]:
daily_ops = enrolment.groupby(['date', 'district'])['Total_Enrolments'].sum().reset_index()
daily_ops['date_int'] = daily_ops['date'].astype(int) / 10**9 # Convert to numeric for ML

# ML: Isolation Forest
model = IsolationForest(contamination=0.005, random_state=42)
daily_ops['anomaly'] = model.fit_predict(daily_ops[['Total_Enrolments']])
anomalies = daily_ops[daily_ops['anomaly'] == -1]

print(f"INTEGRITY ALERT: {len(anomalies)} suspicious operational spikes detected.")

fig = px.scatter(daily_ops, x='date', y='Total_Enrolments',
                 color=daily_ops['anomaly'].astype(str),
                 color_discrete_map={'-1': 'red', '1': 'blue'},
                 hover_name='district',
                 title='Integrity Shield: Detecting Operational Fraud (Red = Abnormal Spike)',
                 template='plotly_dark',
                 opacity=0.6)
fig.show()

## 5. 🎓 Youth Awareness Tracker (The Missing 15-Year-Olds)
**Insight**: Analyzing the correlation between Age 5-17 New Enrolments vs. Biometric Updates.
**Visualization**: Correlation Heatmap.

In [ ]:
youth_bio = biometric.groupby(['state', 'district'])['bio_age_5_17'].sum().reset_index()
youth_enrol = enrolment.groupby(['state', 'district'])['age_5_17'].sum().reset_index()
youth_metrics = pd.merge(youth_bio, youth_enrol, on=['state', 'district'])
youth_metrics['Update_Ratio'] = youth_metrics['bio_age_5_17'] / (youth_metrics['age_5_17'] + 1)

# Filter Top 50 Districts by Volume for cleaner correlation chart
top_districts = youth_metrics.nlargest(50, 'age_5_17')

fig = px.bar(top_districts, x='district', y=['bio_age_5_17', 'age_5_17'],
             barmode='group',
             title='Youth Tracker: Enrolment (Red) vs Biometric Updates (Blue) Gap',
             template='plotly_dark')
fig.show()